In [11]:
%reload_ext autoreload

%autoreload 2
import numpy as np
from models import lstm
import torch

In [12]:
# FILES TO CHANGE (This is where your sequence_nlp_harvey.zip etc is) 

NPY_INPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/irma/'

In [13]:
def test_100(X_train, y_train, X_test, y_test):
    running_acc = 0.0
    running_auc = 0.0
    graph_results = []
    bin_names = []
    
    
    for i in range(100):
        model = lstm.LSTM(input_dim = 400, hidden_dim = 135, num_layers = 1, bidirectional = True,
                  learning_rate = 0.03151218903790648, momentum = 0.048646448029555533, decay_factor = 0.12137002758543824)
        model.learn(X_train, y_train, epochs=2)
        
        # evaluate
        running_acc += model.get_accuracy(torch.Tensor(X_test), torch.Tensor(y_test))
        running_auc += model.get_auc(torch.Tensor(X_test), torch.Tensor(y_test))
        bins, accRates = model.get_accuracy_graph(torch.Tensor(X_test), torch.Tensor(y_test))
        bin_names.append(bins)
        graph_results.append(accRates)
    model.plot_CIs(bin_names, graph_results)
    
    return running_acc/100, running_auc/100

In [ ]:
# load data from files

X_seq_tfidf_train = np.load(NPY_INPUT_DIR + 'X_seq_tfidf_train.npz')
X_seq_tfidf_test = np.load(NPY_INPUT_DIR + 'X_seq_tfidf_test.npz')


X_seq_glove_train = np.load(NPY_INPUT_DIR + 'X_seq_glove_train.npz')
X_seq_glove_test = np.load(NPY_INPUT_DIR + 'X_seq_glove_test.npz')

y_train = torch.Tensor(np.load(NPY_INPUT_DIR + 'y_train.npy'))
y_test = torch.Tensor(np.load(NPY_INPUT_DIR + 'y_test.npy'))

In [14]:
# format npz objects as lists of arrays again
X_tfidf_train = [ torch.Tensor(X_seq_tfidf_train[file]) for file in X_seq_tfidf_train.files]
X_tfidf_test = [ torch.Tensor(X_seq_tfidf_test[file]) for file in X_seq_tfidf_test.files]

X_glove_train = [ torch.Tensor(X_seq_glove_train[file]) for file in X_seq_glove_train.files]
X_glove_test = [ torch.Tensor(X_seq_glove_test[file]) for file in X_seq_glove_test.files]

In [15]:
# build lists with features concatenated

from sklearn.preprocessing import StandardScaler

X_raw_train = []
for i in range(len(y_train)):
    file = X_seq_tfidf_train.files[i]
    X_raw_train.append(np.concatenate((X_seq_tfidf_train[file], X_seq_glove_train[file]), axis=1))

scaler = StandardScaler()
X_fit_train = np.concatenate(X_raw_train)
scaler.fit(X_fit_train)

X_train = [torch.Tensor(scaler.transform(X_i)) for X_i in X_raw_train]

X_test = []
for i in range(len(y_test)):
    file = X_seq_tfidf_test.files[i]
    X_test.append(torch.Tensor(scaler.transform(np.concatenate((X_seq_tfidf_test[file], X_seq_glove_test[file]), axis=1))))


In [16]:
%%capture
# mutes output for this cell

acc, auc = test_100(X_train,y_train,X_test,y_test)

epoch: 0 learning rate: [0.03151218903790648]
epoch: 1 learning rate: [0.003824635252808254]


In [ ]:
print(acc)
print(auc)
